---
title: "Manipulating Geospatial Data (starbucks)"
author: "Jungwoo Lee"
date: "2023-05-29"
categories: [code, python]
image: "kiva.png"
---

> Manipulating Geospatial Data Exercise, Kaggle

# Introduction

당신은 스타벅스 리저브 로스터리의 다음 매장을 찾고 있는 스타벅스 빅 데이터 분석가([그게 진짜 직업입니다!](https://www.forbes.com/sites/bernardmarr/2018/05/28/starbucks-using-big-data-analytics-and-artificial-intelligence-to-boost-performance/#130c7d765cdc)))입니다.
이 [로스터리](https://www.businessinsider.com/starbucks-reserve-roastery-compared-regular-starbucks-2018-12#also-on-the-first-floor-was-the-main-coffee-bar-five-hourglass-like-units-hold-the-freshly-roasted-coffee-beans-that-are-used-in-each-order-the-selection-rotates-seasonally-5)는 일반적인 스타벅스 매장보다 훨씬 크며 고급 라운지 공간과 함께 다양한 음식 및 와인 옵션을 포함한 몇 가지 추가 기능을 갖추고 있습니다. 잠재적으로 적합한 위치를 결정하기 위해 캘리포니아 주에 있는 다양한 카운티의 인구 통계를 조사합니다.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/BIyE6kR.png" width="450"><br/><br/>
</center>

시작하기 전에 아래 코드 셀을 실행하여 모든 것을 설정하십시오.

In [11]:
import math
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim 
from geopandas.tools import geocode            # What you'd normally run
# from learntools.geospatial.tools import Nominatim # Just for this exercise

import folium 
from folium import Marker
from folium.plugins import MarkerCluster

# from learntools.core import binder
# binder.bind(globals())
# from learntools.geospatial.ex4 import *

이전 연습의 `embed_map()` 함수를 사용하여 지도를 시각화합니다.

In [5]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Exercises

### 1) Geocode the missing locations.

다음 코드 셀을 실행하여 캘리포니아 주의 스타벅스 위치를 포함하는 DataFrame `starbucks`를 생성합니다.


In [6]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("/Users/jungwoolee/Desktop/college/data mining/geo_data/starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


대부분의 상점에는 알려진 위치(위도, 경도)가 있습니다. 그러나 Berkeley 시의 모든 위치가 누락되었습니다.

In [7]:
# How many rows in each column have missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


아래 코드 셀을 사용하여 Nominatim 지오코더로 이러한 값을 채웁니다.

튜토리얼에서 우리는 `Nominatim()`(`geopy.geocoders`에서)을 사용하여 값을 지오코딩했으며 이는 이 과정 외부에서 자신의 프로젝트에서 사용할 수 있는 것입니다.

이 연습에서는 약간 다른 함수 `Nominatim()`(`learntools.geospatial.tools`에서)을 사용합니다. 이 기능은 노트북 상단에서 가져온 기능으로 GeoPandas의 기능과 동일하게 작동합니다.

즉, 다음과 같은 경우:
- 노트북 상단의 import 문을 변경하지 않고
- 아래 코드 셀에서 `geocode()`로 지오코딩 함수를 호출합니다.

코드가 의도한 대로 작동합니다!


In [15]:
# Create the geocoder
geolocator = Nominatim(user_agent="kaggle_learn")

# Your code here
def my_geocoder(row):
    point = geocode(row, provider='nominatim').geometry[0]
    return pd.Series({'Longitude': point.x, 'Latitude': point.y})

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)

ConfigurationError: Using Nominatim with default or sample `user_agent` "geopy/2.3.0" is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`.